In [44]:
!pip install DeepXDE==0.10

gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Select the Runtime > "Change runtime type" menu to enable a GPU accelerator, ')
  print('and then re-execute this cell.')
else:
  print(gpu_info)

/bin/bash: nvidia-smi: command not found


In [54]:
import sys
import os
#dir_path = os.path.dirname(os.path.realpath(__file__))
#sys.path.append(dir_path)
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import argparse
import scipy.io
import numpy as np
import deepxde as dde # version 0.11 or higher
from deepxde.backend import tf

In [46]:
## Network Parameters
# 1D
input_1d = 2 # network input size (1D) # dimension of the X
num_hidden_layers_1d = 4 # number of hidden layers for NN (1D)
hidden_layer_size_1d = 32 # size of each hidden layers (1D)
output_1d = 3 # network output size (1D)
# 2D
input_2d = 3 # network input size (2D)
num_hidden_layers_2d = 4 # number of hidden layers for NN (2D)
hidden_layer_size_2d = 32 # size of each hidden layers (2D)
output_2d = 3 # network output size (2D)
output_heter = 3 # network output size for heterogeneity case (2D)
## Training Parameters
num_domain = 20000 # number of training points within the domain
num_boundary = 1000 # number of training boundary condition points on the geometry boundary
num_test = 1000 # number of testing points within the domain
MAX_MODEL_INIT = 24 #16 # maximum number of times allowed to initialize the model
MAX_LOSS = 4 # upper limit to the initialized loss
epochs = 60000 #60000 # number of epochs for training
lr =  0.0005 # learning rate
noise = 0 #0.1 # noise factor
test_size = 0.9 # precentage of testing data

In [145]:
## FentonKarma additions

pars = scipy.io.loadmat("FKpars2.mat")
ini = pars["ini"]

# print(np.shape(ini))
nset = 6 

uv=float(ini[nset-1,12-1])
uw=uv
uu=uw
uvsi=float(ini[nset-1,13-1])
ucsi=float(ini[nset-1,11-1])
k=float(ini[nset-1,10-1])
taud=float(ini[nset-1,6-1])
tauv2=float(ini[nset-1,3-1])
tauv1=float(ini[nset-1,2-1])
tauvplus=float(ini[nset-1,1-1])
tauo=float(ini[nset-1,7-1])
tauwminus=float(ini[nset-1,5-1])
tauwplus=float(ini[nset-1,4-1])
taur=float(ini[nset-1,8-1])
tausi=float(ini[nset-1,9-1])

D=0.0500

In [136]:
filename = "set6forPINNs.mat"
dim = 2
data = scipy.io.loadmat(filename)
if dim == 1:
    T, X, usav, u0, x0 = data["T"], data["X"], data["ulin"], data["u0"], data["x0"]
elif dim == 2:
    T, X, Y, usav, u0, x0 = data["T"], data["X"], data["Y"], data["ulin"], data["u0"], data["x0"]

min_x = np.amin(X)
max_x = np.amax(X)
min_t = np.amin(T)
max_t = np.amax(T)
spacing = X[2]-X[1]
if dim == 2:
    min_y = np.amin(Y)
    max_y = np.amax(Y)
    observe_x=np.concatenate((X,Y,T),axis=1)
elif dim==1:
    observe_x=np.concatenate((X,T),axis=1)
    
print(np.shape(observe_x))

(2000000, 3)


In [137]:
def pde_1D(x, y):
    u, v, w = y[:, 0:1], y[:, 1:2], y[:, 2:3]
    dv_dt = dde.grad.jacobian(v, x, i=0, j=1)
    du_dt = dde.grad.jacobian(u, x, i=0, j=1)
    du_dxx = dde.grad.hessian(y, x, component=0, i=0, j=0)
    dw_dt = dde.grad.jacobian(w, x, i=0, j=1)

    tauv = tf.cast(tf.math.less_equal(uvsi, u),tf.float32)*tauv2 + tf.cast(tf.math.less_equal(u, uvsi),tf.float32)*tauv1
    tauv = tf.cast(tf.math.less_equal(u, uv),tf.float32)*tauv + tf.cast(tf.math.less_equal(uv, u),tf.float32)*tauvplus
    vinf = tf.cast(tf.math.less_equal(u, uv),tf.float32)
    Fu = tf.cast(tf.math.less_equal(uv,u),tf.float32)*((u-uv)*(tf.ones([1],tf.float32)-u))
    Jfi = Fu*(-v) / taud  
    Uu = tf.cast(tf.math.less_equal(uu, u),tf.float32) + tf.cast(tf.math.less_equal(u, uu),tf.float32)*u
    tauu = tf.cast(tf.math.less_equal(uu, u),tf.float32)*taur + tf.cast(tf.math.less_equal(u, uu),tf.float32)*tauo
    Jso = Uu/tauu
    winf = tf.cast(tf.math.less_equal(u, uw),tf.float32) + tf.cast(tf.math.less_equal(uw, u),tf.float32)*0
    tauw = tf.cast(tf.math.less_equal(u, uw),tf.float32)*tauwminus + tf.cast(tf.math.less_equal(uw, u),tf.float32)*tauwplus
    Jsi = -w/tausi/2*(tf.ones([1],tf.float32) + tf.nn.tanh(k*(u-ucsi)))

    Iion = -(Jfi + Jsi + Jso) 

    eq_a = du_dt - (Iion+D*du_dxx)
    eq_b = dv_dt - (vinf - v) / tauv
    eq_c = dw_dt - (winf - w) / tauw

    return [eq_a, eq_b, eq_c]

In [150]:
def pde_2D(x, y):
#     y=tf.cast(y,tf.float32)
    u, v, w = y[:, 0:1], y[:, 1:2], y[:, 2:3]
#     x=tf.cast(x,tf.float32)
    dv_dt = dde.grad.jacobian(v, x, i=0, j=1)
    du_dt = dde.grad.jacobian(u, x, i=0, j=1)
    du_dxx = dde.grad.hessian(y, x, component=0, i=0, j=0)
    du_dyy = dde.grad.hessian(y, x, component=0, i=1, j=1)
    dw_dt = dde.grad.jacobian(w, x, i=0, j=1)
    
    tauv = tf.cast(tf.math.less_equal(uvsi, u),tf.float32)*tauv2 + tf.cast(tf.math.less_equal(u, uvsi),tf.float32)*tauv1
    tauv = tf.cast(tf.math.less_equal(u, uv),tf.float32)*tauv + tf.cast(tf.math.less_equal(uv, u),tf.float32)*tauvplus
    vinf = tf.cast(tf.math.less_equal(u, uv),tf.float32)
    Fu = tf.cast(tf.math.less_equal(uv,u),tf.float32)*((u-uv)*(tf.ones([1],tf.float32)-u))
    Jfi = Fu*(-v) / taud  
    Uu = tf.cast(tf.math.less_equal(uu, u),tf.float32) + tf.cast(tf.math.less_equal(u, uu),tf.float32)*u
    tauu = tf.cast(tf.math.less_equal(uu, u),tf.float32)*taur + tf.cast(tf.math.less_equal(u, uu),tf.float32)*tauo
    Jso = Uu/tauu
    winf = tf.cast(tf.math.less_equal(u, uw),tf.float32) + tf.cast(tf.math.less_equal(uw, u),tf.float32)*0
    tauw = tf.cast(tf.math.less_equal(u, uw),tf.float32)*tauwminus + tf.cast(tf.math.less_equal(uw, u),tf.float32)*tauwplus
    Jsi = -w/tausi/2*(tf.ones([1],tf.float32) + tf.nn.tanh(k*(u-ucsi)))


    Iion = -(Jfi + Jsi + Jso) 

    eq_a = du_dt - (Iion+D*(du_dxx+du_dyy))
    eq_b = dv_dt - (vinf - v) / tauv
    eq_c = dw_dt - (winf - w) / tauw

    return [eq_a, eq_b, eq_c]

In [151]:
## Split into train and test data
observe_train, observe_test, u_train, u_test = train_test_split(observe_x, usav, test_size=test_size)
u_train = u_train + noise*np.random.randn(u_train.shape[0], u_train.shape[1])

## Geometry and Time domains
if dim == 1:
    geom = dde.geometry.Interval(min_x, max_x)
    timedomain = dde.geometry.TimeDomain(min_t, max_t)
    geomtime = dde.geometry.GeometryXTime(geom, timedomain)
elif dim == 2:
    geom = dde.geometry.Rectangle([min_x,min_y], [max_x,max_y])
    timedomain = dde.geometry.TimeDomain(min_t, max_t)
    geomtime = dde.geometry.GeometryXTime(geom, timedomain)
    
## Define Boundary Conditions
if dim == 1:
    bc = dde.NeumannBC(geomtime, lambda x:  np.zeros((len(x), 1)), lambda _, on_boundary: on_boundary, component=0)
elif dim == 2:
    def boundary_func_2d(x, on_boundary):
        return on_boundary and ~(x[0:2]==[min_x,min_y]).all() and  ~(x[0:2]==[min_x,max_y]).all() and ~(x[0:2]==[max_x,min_y]).all()  and  ~(x[0:2]==[max_x,max_y]).all()
    bc = dde.NeumannBC(geomtime, lambda x:  np.zeros((len(x), 1)), boundary_func_2d, component=0)

    ## Define Initial Conditions
ic1 = dde.PointSetBC(x0,u0,component=0)
# Consider adding the below
# ic2 = dde.PointSetBC(observe_init,v_init,component=1)
# ic3 = dde.PointSetBC(observe_init,w_init,component=2)

## Model observed data
observe_u = dde.PointSetBC(observe_train, u_train, component=0)
input_data = [bc, ic1, observe_u]

In [153]:
## Select relevant PDE (Dim, Heterogeneity) and define the Network
if dim == 1:
    pde = pde_1D
    net = dde.maps.FNN([input_1d] + [hidden_layer_size_1d] * num_hidden_layers_1d + [output_1d], "tanh", "Glorot uniform")
elif dim == 2:
    pde = pde_2D
    net = dde.maps.FNN([input_2d] + [hidden_layer_size_2d] * num_hidden_layers_2d + [output_2d], "tanh", "Glorot uniform")
pde_data = dde.data.TimePDE(geomtime, pde, input_data,
                        num_domain = num_domain,
                        num_boundary=num_boundary,
                        anchors=observe_train,
                        num_test=num_test)
model = dde.Model(pde_data, net)

In [ ]:


## Stabilize initialization process by capping the losses

initial_loss =10e12
num_init = 0
while initial_loss>MAX_LOSS or np.isnan(initial_loss).any() or np.isinf(initial_loss).any():  # added checking for inf values
    num_init += 1
    model = dde.Model(pde_data, net)
    model.compile("adam", lr=lr)
    #model.compile("adam", lr=lr, loss_weights=loss_weights)
    losshistory, _ = model.train(epochs=1)
#     losshistory, train_state = model.train(epochs=epochs, model_save_path = out_path, display_every=1000)
    initial_loss = max(losshistory.loss_train[0])
    if num_init > MAX_MODEL_INIT:
        raise ValueError('Model initialization phase exceeded the allowed limit')

losshistory, train_state = model.train(epochs=epochs)  # crashes here

# Plot loss history
loss_train = np.sum(losshistory.loss_train, axis=1)
loss_test = np.sum(losshistory.loss_test, axis=1)


Compiling model...
'compile' took 0.828907 s

Initializing variables...
Training model...

0         [1.56e-01, 4.11e-03, 1.47e-04, 2.01e-04, 1.49e-01, 1.18e-01]    [1.28e-01, 3.25e-03, 1.26e-04, 2.01e-04, 1.49e-01, 1.18e-01]    []  
1         [1.22e-01, 3.66e-03, 1.46e-04, 2.06e-04, 1.34e-01, 1.13e-01]    [8.73e-02, 2.87e-03, 1.25e-04, 2.06e-04, 1.34e-01, 1.13e-01]    []  

Best model at step 1:
  train loss: 3.73e-01
  test loss: 3.38e-01
  test metric: []

'train' took 6.205405 s

Training model...

Step      Train loss                                                      Test loss                                                       Test metric
1         [1.22e-01, 3.66e-03, 1.46e-04, 2.06e-04, 1.34e-01, 1.13e-01]    [8.73e-02, 2.87e-03, 1.25e-04, 2.06e-04, 1.34e-01, 1.13e-01]    []  
1000      [4.88e-04, 5.60e-05, 3.92e-06, 1.12e-03, 5.93e-02, 7.20e-02]    [2.98e-04, 4.93e-05, 3.01e-06, 1.12e-03, 5.93e-02, 7.20e-02]    []  
2000      [3.86e-04, 5.93e-05, 2.49e-06, 1.01e-03, 4.53e

In [ ]:
model_pred = model.predict(observe_test)
upred = model_pred[:,0:1]
rmse_v = np.sqrt(np.square(upred - usav).mean())
print('rmse_v:', rmse_v)

dici1={"model_pred": model_pred, "observe_x": observe_x, "ulin": usav}
file_name_save = file_name[0:-4] + "DONE.mat"
spy.savemat(file_name_save,dici1)